In [4]:
pip install langchain-huggingface

Defaulting to user installation because normal site-packages is not writeable
  Using cached langchain_huggingface-1.2.0-py3-none-any.whl.metadata (2.8 kB)
Using cached langchain_huggingface-1.2.0-py3-none-any.whl (30 kB)
   ---------------------------------------- 0.0/566.4 kB ? eta -:--:--
   ------------------ --------------------- 262.1/566.4 kB ? eta -:--:--
   ---------------------------------------- 566.4/566.4 kB 7.1 MB/s  0:00:00

  Attempting uninstall: huggingface-hub

    Found existing installation: huggingface_hub 1.4.1

   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
    Uninstalling huggingface_hub-1.4.1:
   ---------------------------------------- 0/2 [huggingface-hub]
      Successfully uninstalled huggingface_hub-1.4.1
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------------- 0/2 [huggingface-hub]
   ---------------------------------

The system cannot find the path specified.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.1.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.2 which is incompatible.


In [2]:
pip install sentence_transformers faiss-cpu transformers 


Defaulting to user installation because normal site-packages is not writeable
  Using cached huggingface_hub-1.4.1-py3-none-any.whl.metadata (13 kB)
Using cached huggingface_hub-1.4.1-py3-none-any.whl (553 kB)
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 0.36.2
    Uninstalling huggingface_hub-0.36.2:
      Successfully uninstalled huggingface_hub-0.36.2
Note: you may need to restart the kernel to use updated packages.


The system cannot find the path specified.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-huggingface 1.2.0 requires huggingface-hub<1.0.0,>=0.33.4, but you have huggingface-hub 1.4.1 which is incompatible.


In [1]:
"""
Task 10 & 11 - POC 1
Using RAG, build an e-commerce assistant for product recommendations and Q&A
LLM: Ollama (llama3)
"""

from langchain_ollama import ChatOllama
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate


# -------------------------------------------------
# Load product data
# -------------------------------------------------
loader = TextLoader("products.txt")
documents = loader.load()

# -------------------------------------------------
# Chunk the data
# -------------------------------------------------
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200,
    chunk_overlap=50
)
chunks = text_splitter.split_documents(documents)

# -------------------------------------------------
# Create embeddings (open-source)
# -------------------------------------------------
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# -------------------------------------------------
# Store embeddings in FAISS
# -------------------------------------------------
vectorstore = FAISS.from_documents(
    chunks,
    embeddings
)

retriever = vectorstore.as_retriever(
    search_kwargs={"k": 3}
)

# -------------------------------------------------
# Create LLM
# -------------------------------------------------
llm = ChatOllama(
    model="llama3",
    temperature=0
)

# -------------------------------------------------
# RAG Prompt
# -------------------------------------------------
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an e-commerce assistant. "
            "Use the given context to answer the question."
        ),
        ("human", "Context:\n{context}\n\nQuestion:\n{question}")
    ]
)

# -------------------------------------------------
# RAG Pipeline
# -------------------------------------------------
def ask_question(question: str):
    docs = retriever.invoke(question)

    context = "\n\n".join(doc.page_content for doc in docs)

    chain = prompt | llm
    response = chain.invoke(
        {
            "context": context,
            "question": question
        }
    )
    return response.content

# -------------------------------------------------
# Test queries
# -------------------------------------------------
if __name__ == "__main__":
    questions = [
        "Recommend a good smartphone under 70000",
        "Which laptop is best for portability?",
        "Suggest noise cancelling headphones",
    ]

    for q in questions:
        print(f"\nQuestion: {q}")
        answer = ask_question(q)
        print("Answer:", answer)


Loading weights:   0%|          | 0/103 [00:00<?, ?it/s]

BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.



Question: Recommend a good smartphone under 70000
Answer: Based on the given products, I would recommend the Samsung Galaxy S23 as it falls within your budget of ₹70,000. It's an Android flagship with a Snapdragon processor and AMOLED display, which are great features for a smartphone. Additionally, its price is lower than the iPhone 14, making it a more affordable option.

Question: Which laptop is best for portability?
Answer: Based on the given context, I would recommend the Dell XPS 13 or the Apple MacBook Air M2 as both are lightweight laptops. However, if you're looking for the absolute most portable option, I would suggest the Apple MacBook Air M2.

According to its description, it is "Thin and powerful", which implies that it is designed with portability in mind. Additionally, its price point of ₹1,15,000 is relatively competitive compared to other laptops in this category.

The Dell XPS 13 is also a lightweight laptop, but it's slightly more expensive at ₹1,20,000. While it s